# Calculating maximum yield of flaviolin in P. putida 

Here we will use a genome-scale model to predict the maximum yield of flaviolin in P. putida. 

This notebook has been tested on [jprime.lbl.gov](jprime.lbl.gov) with the **biodesign_3.7** kernel.

# Setup

In [1]:
import cobra
from cobra.flux_analysis import flux_variability_analysis

import numpy as np
from scipy.stats import norm
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from IPython.display import Image                                                 # Used to display images

# Getting and preparing the genome-scale model

In [2]:
OUTPUT_DIR = '../data/flaviolin'

## Load model

In [3]:
file_name = '../data/iJN1463_modifiedv2.json'
model = cobra.io.load_json_model(file_name)   # Model from BIGG slightly improved by Joonhoon
cobra.io.write_sbml_model(model, f'{OUTPUT_DIR}/iJN1463_modified.xml')

## Add pathway to model

The model already has all the reactions up to malonyl-CoA:

In [4]:
model.reactions.ACCOAC

Reaction identifier,ACCOAC
Name,Acetyl-CoA carboxylase
Memory address,0x07f5c45ef9ad0
Stoichiometry,accoa_c + atp_c + hco3_c --> adp_c + h_c + malcoa_c + pi_c Acetyl-CoA + ATP C10H12N5O13P3 + Bicarbonate --> ADP C10H12N5O10P2 + H+ + Malonyl CoA C24H33N7O19P3S + Phosphate
GPR,PP_1607 and PP_0559 and PP_0558 and PP_1996
Lower bound,0.0
Upper bound,1000.0


### Exogenous pathway

<span style="color:red">"Flaviolin is a spontaneous oxidation product of 1,3,6,8-tetrahydroxynaphthalene (THN)... The S. griseus rppA gene encodes RppA... which is sufficient in vitro for the synthesis of THN from malonyl-CoA." (https://pubmed.ncbi.nlm.nih.gov/12028378/)
</span>

**1-)** malonyl-CoA to 1,3,6,8-THN

Create THN metabolite first:

In [5]:
thn_c = cobra.Metabolite(
    'thn_c',
    formula='C10H8O4',
    name='1,3,6,8-Naphthalenetetrol',
    compartment='c')
thn_c.charge = 0

And now creating the reaction and adding it to the model:

https://www.uniprot.org/uniprot/B0FYK7

In [6]:
THNS = cobra.Reaction('THNS')
THNS.name = 'tetrahydroxynaphthalene synthase'
THNS.subsystem = 'flaviolin biosynthesis'
THNS.lower_bound = 0.  # This is the default
THNS.upper_bound = 1000.  # This is the default

In [7]:
mal_c = model.metabolites.get_by_id('malcoa_c')
co2_c = model.metabolites.get_by_id('co2_c')
coa_c = model.metabolites.get_by_id('coa_c')
h_c = model.metabolites.get_by_id('h_c')
h2o_c = model.metabolites.get_by_id('h2o_c')

THNS.add_metabolites({
    mal_c: -5.0,
    h_c: -5,
    thn_c: 1.0,
    co2_c: 5.0,
    coa_c: 5.0,
    h2o_c: 1
})
h_c.charge=1
THNS.gene_reaction_rule = 'rppA'

In [8]:
THNS.check_mass_balance()

{}

In [9]:
model.add_reactions([THNS])

In [10]:
model.reactions.THNS

Reaction identifier,THNS
Name,tetrahydroxynaphthalene synthase
Memory address,0x07f5b95493450
Stoichiometry,"5 h_c + 5.0 malcoa_c --> 5.0 co2_c + 5.0 coa_c + h2o_c + thn_c 5 H+ + 5.0 Malonyl CoA C24H33N7O19P3S --> 5.0 CO2 CO2 + 5.0 Coenzyme A + H2O H2O + 1,3,6,8-Naphthalenetetrol"
GPR,rppA
Lower bound,0.0
Upper bound,1000.0


In [11]:
model.metabolites.get_by_id('thn_c') 

Metabolite identifier,thn_c
Name,"1,3,6,8-Naphthalenetetrol"
Memory address,0x07f5b95508510
Formula,C10H8O4
Compartment,c
In 1 reaction(s),THNS


**2-)** THN oxidation to flaviolin:

Creating flaviolin metabolite first:

https://pubchem.ncbi.nlm.nih.gov/compound/Flaviolin

In [12]:
flavio_c = cobra.Metabolite(
    'flavio_c',
    formula='C10H6O5',
    name='Flaviolin',
    compartment='c')
flavio_c.charge = 0

And now creating the reaction and adding it to the model:

In [13]:
FLAS = cobra.Reaction('FLAS')
FLAS.name = 'tetrahydroxynaphthalene oxidation'
FLAS.subsystem = 'flaviolin biosynthesis'
FLAS.lower_bound = 0.  # This is the default
FLAS.upper_bound = 1000.  # This is the default

https://biocyc.org/META/NEW-IMAGE?type=REACTION&object=RXN-9930

In [14]:
o2_c = model.metabolites.o2_c
FLAS.add_metabolites({
    o2_c: -1.0,
    thn_c: -1.0,
    flavio_c: 1.0,
    h2o_c: 1.0
})

In [15]:
FLAS.check_mass_balance()

{}

In [16]:
model.add_reactions([FLAS])
model.reactions.FLAS

Reaction identifier,FLAS
Name,tetrahydroxynaphthalene oxidation
Memory address,0x07f5b954c3b10
Stoichiometry,"o2_c + thn_c --> flavio_c + h2o_c O2 O2 + 1,3,6,8-Naphthalenetetrol --> Flaviolin + H2O H2O"
GPR,
Lower bound,0.0
Upper bound,1000.0


In [17]:
model.add_boundary(flavio_c, type='demand')

Reaction identifier,DM_flavio_c
Name,Flaviolin demand
Memory address,0x07f45822d3c10
Stoichiometry,flavio_c --> Flaviolin -->
GPR,
Lower bound,0
Upper bound,1000.0


## Second option - transport to cell exterior and exchange reaction for flaviolin (same result)

Then transport from cytosol to exterior:

In [17]:
flavio_e = cobra.Metabolite(
    'flavio_e',
    formula='C10H6O5',
    name='Flaviolin',
    compartment='e')


In [19]:
flavioex = cobra.Reaction('FLAEX')
flavioex.name = 'Flaviolin transport extracellular'
flavioex.subsystem = 'Flaviolin Biosynthesis'
flavioex.lower_bound = 0.     # This is the default
flavioex.upper_bound = 1000.  # This is the default

In [20]:
flavioex.add_metabolites({
    flavio_c: -1.0,
    flavio_e: 1.0,
})
flavioex.gene_reaction_rule = ' '

In [21]:
flavioex.check_mass_balance()

{}

And finally the exchange reaction:

In [26]:
EX_flavio_e = cobra.Reaction('EX_flavio_e')
EX_flavio_e.name = 'Flaviolin exchange'
EX_flavio_e.subsystem = 'Flaviolin Biosynthesis'
EX_flavio_e.lower_bound = 0.  # This is the default
EX_flavio_e.upper_bound = 1000.  # This is the default

In [27]:
EX_flavio_e.add_metabolites({
    flavio_e: -1.0,
})
EX_flavio_e.gene_reaction_rule = ' '

In [28]:
model.add_reactions([flavioex, EX_flavio_e])

In [29]:
model.reactions.get_by_id('FLAEX')

Reaction identifier,FLAEX
Name,Flaviolin transport extracellular
Memory address,0x07f5c52ee4310
Stoichiometry,flavio_c --> flavio_e Flaviolin --> Flaviolin
GPR,
Lower bound,0.0
Upper bound,1000.0


In [30]:
model.reactions.get_by_id('EX_flavio_e')

Reaction identifier,EX_flavio_e
Name,Flaviolin exchange
Memory address,0x07f5b9540b590
Stoichiometry,flavio_e --> Flaviolin -->
GPR,
Lower bound,0.0
Upper bound,1000.0


In [31]:
model.metabolites.get_by_id('flavio_e')

Metabolite identifier,flavio_e
Name,Flaviolin
Memory address,0x07f5b954644d0
Formula,C10H6O5
Compartment,e
In 2 reaction(s),"EX_flavio_e, FLAEX"


In [ ]:
cobra.io.save_json_model(model, "iJN1463_mod_flavio.json")
cobra.io.write_sbml_model(model, "iJN1463_mod_flavio.xml")

## Find maximum yield

Set model objetive to the exchange reaction for the final product (so as to maximize its production):

In [32]:
# model.objective = model.reactions.DM_flavio_c
model.objective = model.reactions.EX_flavio_e

In [33]:
# model.reactions.DM_flavio_c
model.reactions.EX_flavio_e

Reaction identifier,EX_flavio_e
Name,Flaviolin exchange
Memory address,0x07f5b9540b590
Stoichiometry,flavio_e --> Flaviolin -->
GPR,
Lower bound,0.0
Upper bound,1000.0


Let's optimize product formation:

In [34]:
solution = model.optimize()

The maximum production in mMol/gdw/hr for flaviolin is:

In [21]:
solution.fluxes['DM_flavio_c']
# solution.fluxes['EX_flavio_e']

2.7305945945945953

In [35]:
# solution.fluxes['DM_flavio_c']
solution.fluxes['EX_flavio_e']

2.730594594594593

This number increases with the glucose input:

In [36]:
solution.fluxes['EX_glc__D_e']

-6.0

but the ratio (i.e. the maximum theoretical yield) is constant:

In [23]:
abs(solution.fluxes['DM_flavio_c']/solution.fluxes['EX_glc__D_e'])

0.45509909909909924

In [37]:
abs(solution.fluxes['EX_flavio_e']/solution.fluxes['EX_glc__D_e'])

0.45509909909909885

For our 20mM glucose, this corresponds to 9mM of flaviolin, i.e. 1.8 g/L